In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

# Import scripts
from script_files.feature_engineering import *
from script_files.prepare_flight_data import *
from script_files.dummies import *

RSEED = 42

In [2]:
train_df = pd.read_csv('data/final_train.csv', index_col=[0])
train_df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,delayed,domestic,dep_hour,dep_weekday,duration_min,arr_hour,flight_month,flight_month_name,year,distance
0,train_id_15674,2016-01-01 00:00:00,TU 0564,NKC,TUN,2016-01-01 00:15:00,2016-01-01 04:30:00,ATA,TU 320IMV,0.0,...,0,0,0,Friday,255.0,4,1,January,2016,3298.067996
1,train_id_15676,2016-01-01 00:00:00,TU 0714,JED,TUN,2016-01-01 00:55:00,2016-01-01 05:30:00,ATA,TU 332IFM,195.0,...,1,0,0,Friday,275.0,5,1,January,2016,3256.052105
2,train_id_15675,2016-01-01 00:00:00,TU 0614,DKR,TUN,2016-01-01 01:20:00,2016-01-01 05:55:00,ATA,TU 320IMU,49.0,...,1,0,1,Friday,275.0,5,1,January,2016,3678.974557
3,train_id_30980,2016-01-01 00:00:00,UG 0002,TUN,DJE,2016-01-01 06:15:00,2016-01-01 07:15:00,SCH,UG AT7LBD,0.0,...,0,1,6,Friday,60.0,7,1,January,2016,333.916459
4,train_id_7179,2016-01-01 00:00:00,TU 0880,TUN,AMS,2016-01-01 06:30:00,2016-01-01 09:20:00,ATA,TU 736IOP,36.0,...,1,0,6,Friday,170.0,9,1,January,2016,1770.371959


In [3]:
train_df.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC', 'target', 'icao_DEP', 'iata_DEP', 'name_DEP', 'city_DEP',
       'subd_DEP', 'country_DEP', 'elevation_DEP', 'lat_DEP', 'lon_DEP',
       'tz_DEP', 'icao_ARR', 'iata_ARR', 'name_ARR', 'city_ARR', 'subd_ARR',
       'country_ARR', 'elevation_ARR', 'lat_ARR', 'lon_ARR', 'tz_ARR',
       'delay_or_onTime', 'delayed', 'domestic', 'dep_hour', 'dep_weekday',
       'duration_min', 'arr_hour', 'flight_month', 'flight_month_name', 'year',
       'distance'],
      dtype='object')

In [4]:
# List of dropping feature
drop_features = ['DATOP', 'ID','FLTID','STD', 'STA','target','icao_DEP', 'iata_DEP', 'name_DEP', 'city_DEP',
       'subd_DEP', 'country_DEP', 'elevation_DEP', 'lat_DEP', 'lon_DEP',
       'tz_DEP', 'icao_ARR', 'iata_ARR', 'name_ARR', 'city_ARR', 'subd_ARR',
       'country_ARR', 'elevation_ARR', 'lat_ARR', 'lon_ARR', 'tz_ARR','arr_hour', 'flight_month','delay_or_onTime', 'delayed']

In [5]:
X_total = drop_column(train_df, drop_features)

In [6]:
train_df

,DEPSTN,ARRSTN,STATUS,AC,domestic,dep_hour,dep_weekday,duration_min,flight_month_name,year,distance
0,NKC,TUN,ATA,TU 320IMV,0,0,Friday,255.0,January,2016,3298.067996
1,JED,TUN,ATA,TU 332IFM,0,0,Friday,275.0,January,2016,3256.052105
2,DKR,TUN,ATA,TU 320IMU,0,1,Friday,275.0,January,2016,3678.974557
3,TUN,DJE,SCH,UG AT7LBD,1,6,Friday,60.0,January,2016,333.916459
4,TUN,AMS,ATA,TU 736IOP,0,6,Friday,170.0,January,2016,1770.371959
...,...,...,...,...,...,...,...,...,...,...,...
107828,TUN,TUN,SCH,TU 32AIMF,1,22,Monday,180.0,December,2018,0.000000
107829,MIR,MIR,SCH,TU 736IOM,1,22,Monday,180.0,December,2018,0.000000
107830,DSS,CKY,ATA,TU 320IMU,0,22,Monday,75.0,December,2018,677.428198
107831,ABJ,NIM,ATA,TU 320IMW,0,22,Monday,115.0,December,2018,1129.691078
